In [ ]:
# |export
#|default_exp p00_titanic_from_scratch


In [ ]:
# |export
import os
import time
import pathlib
import argparse
import torch
from torch import tensor



In [ ]:
# |export
start_time=time.time()
debug=True
_code_git_version="81d57b0fb8f07ffdb767a2e7907efef742253b32"
_code_repository="https://github.com/plops/cl-py-generator/tree/master/example/96_colab_fastai/source/"
_code_generation_time="17:04:19 of Sunday, 2022-08-28 (GMT+1)"
start_time=time.time()
debug=True


In [ ]:
# |export
parser=argparse.ArgumentParser()
parser.add_argument("-v", "--verbose", help="enable verbose output", action="store_true")
args=parser.parse_args()


In [ ]:
# |export
path=pathlib.Path("titanic")
if ( not(path.exists()) ):
    import zipfile
    import kaggle
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f"{path}.zip").extractall(path)


In [ ]:
# |export
import torch
import numpy as np
import pandas as pd
line_char_width=140
np.set_print_options(linewidth=line_char_width)
torch.set_print_options(linewidth=line_char_width, sci_mode=False, edgeitems=7)
pd.set_option("display_width", line_char_width)


In [ ]:
# |export
df=pd.read_csv(((path)/("train.csv")))
df


In [ ]:
df.isna().sum()


In [ ]:
# |export
mode=df.mode().iloc[0]


In [ ]:
# |export
df.fillna(modes, inplace=True)


In [ ]:
df.isna().sum()


In [ ]:
df.describe(include=(np.number,))


In [ ]:
df.Fare.hist()


In [ ]:
# |export
df["LogFare"]=np.log(((1)+(df.Fare)))


In [ ]:
# histogram of logarithm of prices no longer shows the 'long' tail
df.LogFare.hist()


In [ ]:
# look at the three values that are in passenger class. more details about the dataset are here: https://www.kaggle.com/competitions/titanic/data
pclasses=df.Pclass.unique()
pclasses


In [ ]:
# look at columns with non-numeric values
df.describe(include=[object])


In [ ]:
# |export
# replace non-numeric values with numbers by introducing new columns (dummies). The dummy columns will be added to the dataframe df and the 3 original columns are dropped.
# Cabin, Name and Ticket contain too many unique values for this approach to be useful
df=pd.get_dummies(df, columns=["Sex", "Pclass", "Embarked"])
df.columns


In [ ]:
# look at the new dummy columns
added_columns=["Sex_male", "Sex_female", "Pclass_1", "Pclass_2", "Pclass_3", "Embarked_C", "Embarked_Q"]
df[added_columns].head()


In [ ]:
# |export
# create dependent variable as tensor
t_dep=tensor(df.Survived)


In [ ]:
# |export
# independent variables are all continuous variables of interest and the newly created columns
indep_columns=((["Age", "SipSp", "Parch", "LogFare"])+(added_columns))
t_indep=tensor(df[indep_columns].values, dtype=torch.float)
t_indep


In [ ]:
t_indep.shape


In [ ]:
# set up linear model. first we calculate manually a single step for the loss of every row in the dataset. we start with a random coefficient in (-.5,.5) for each column of t_indep
torch.manual_seed(442)
n_coef=t_indep.shape[1]
coefs=((torch.rand(n_coefs))-((0.50    )))
coefs


In [ ]:
# our predictions are formed by multiplying a row with coefficients and summing them up. we don't need to introduce a bias (or intercept) term by introducing a column containing only ones. Such a 'one' is already present in each row in either the dummy column Sex_male or Sex_female.
((t_indep)*(coefs))


In [ ]:
# we have a potential problem with the first column Age. Its values are bigger in average than the values in other columns
# we have two options to normalize Age: 1) divide by maximum or 2) subtract mean and divide by std. In the book jeremy howard uses 1). I will try 2)
if ( False ):
    # method 1)
    vals, indices=t_indep.max(dim=0)
    t_indep=((t_indep)/(vals))
if ( True ):
    # method 2)
    means, indices1=t_indep.mean(dim=0)
    stdts, indices2=t_indep.std(dim=0)
    t_indep=((((t_indep)-(means)))/(stds))
